In [ ]:
%pip install ipykernel

In [ ]:
%pip install langchain-community

In [ ]:
%pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [ ]:
api_key = "5a51b925-734d-4c1d-8d60-930e474158c6"

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

index_name="hybrid-search-langchain-pinecone-1" # in small caps only

## initialize the Pinecone client
pc=Pinecone (api_key=api_key)

In [ ]:
#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
                    name=index_name,
                    dimension=768, #dimension of dense vector by default dimension
                    metric='dotproduct', ## sparse values supportted only for dotproduct
                    spec=ServerlessSpec(cloud='aws', region="us-east-1")
                    )

In [ ]:
index = pc.Index(index_name)
index

In [ ]:
# %pip install langchain-huggingface

In [ ]:
%pip install -qU langchain-ollama

In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [ ]:
# Example text to generate embeddings
sample_text = "This is a test for embeddings."

# Generate the embedding
vector = embeddings.embed_query(sample_text)

# Get the dimension of the embedding
print("Embedding Dimension:", len(vector))

In [ ]:
##  sparse matrix
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

In [ ]:
# %pip install pymupdf

In [ ]:
# Import necessary modules for getting chunked text
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
%pip install "unstructured[md]" 

In [ ]:
%pip install libmagic

In [ ]:
directory_path = r"E:\craw4ai1\extracted_pages"
loader = DirectoryLoader(directory_path, glob="*.md")  # adjust glob pattern for different file types
# Load documents from the directory
documents = loader.load()

In [ ]:
# Set up text splitter for chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

#Split documents into chunks
chunked_docs = text_splitter.split_documents(documents)

In [ ]:
chunked_docs

In [ ]:
# Extract the page_content from each document and store it in a list
contents = [doc.page_content for doc in chunked_docs]
contents

In [ ]:
%pip install nltk

In [ ]:
import nltk
nltk.download('punkt_tab', download_dir='E:\craw4ai1\.venv/nltk_data')

In [ ]:
nltk.data.path.append('E:\craw4ai1\.venv/nltk_data')

In [ ]:
# tfidf values on these sentence
bm25_encoder.fit (contents)

# store the values to a json file 
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object 
bm25_encoder = BM25Encoder().load("bm25_values.json")

In [ ]:
# making hybrid retriever
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)
retriever

In [ ]:
retriever.add_texts(contents)

In [ ]:
User_Query = '''
what was the operating cash flow in 2020?'''

In [ ]:
retrieved_chunks = retriever.invoke(User_Query, k=5)
retrieved_chunks

In [ ]:
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
%pip install -U langchain-ollama

In [ ]:
from langchain_ollama import ChatOllama

In [ ]:
# Initialize the LLM (e.g., OpenAI GPT-3.5 or GPT-4)
llm = ChatOllama(
                            model="deepseek-r1",
                            streaming=True,
                        )

In [ ]:
# 4. Create the Prompt Template for document processing
document_prompt = PromptTemplate(input_variables=["context"], template="Given the following context:\n{context}\nPlease answer the query: {User_Query} concisely:")

In [ ]:
# 5. Set up the StuffDocumentsChain
chain = create_stuff_documents_chain( llm=llm, prompt=document_prompt,)

In [ ]:
%pip install langchain

In [ ]:
# 7. Run the chain with the formatted documents
response = chain.invoke({"context": retrieved_chunks, 'User_Query': User_Query})

In [ ]:
# 8. Output the result
print(response)